# Optuna Evaluator

In this notebook, the results of the Optuna Study are evaluated, visualized in plots and the best hyperparameters are extracted from the hyperparameter tuning with the help of a weighted sum. 

## Notebook Setup

In [2]:
import optuna
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor


## Import Optuna Study

In [7]:

## Change study_name to the name of the run_name of the hyperparameter tuning notebook
study_name = "A6000_OptunaRun_2048"

storage_name = f"sqlite:///optuna/{study_name}.db"
study = optuna.load_study(study_name=study_name, storage=storage_name)

# load study as dataframe
study_results_df = study.trials_dataframe()

In [8]:
## remove failed trial from study
study_results_df.loc[study_results_df.number == 105]

,number,values_0,values_1,values_2,datetime_start,datetime_complete,duration,params_learning_rate,params_lora_alpha_value,params_lora_r_value,params_num_train_epochs,params_warmup_steps,system_attrs_nsga3:generation,state
105,105,NaN,NaN,NaN,2024-04-27 12:15:45.376032,2024-04-27 13:32:43.300230,0 days 01:16:57.924198,0.000033,16,5,8,921,NaN,FAIL


In [9]:
# study_results_df = study_results_df.drop(labels='datetime_start', axis=1)
# study_results_df = study_results_df,drop(labels='datetime_complete', axis=1)
study_results_df = study_results_df.drop(labels='duration', axis=1)
study_results_df = study_results_df.drop(labels='state', axis=1)
study_results_df = study_results_df.drop(labels='system_attrs_nsga3:generation', axis=1)

study_results_df

,number,values_0,values_1,values_2,datetime_start,datetime_complete,params_learning_rate,params_lora_alpha_value,params_lora_r_value,params_num_train_epochs,params_warmup_steps
0,0,0.047838,0.988572,0.049686,2024-04-09 14:26:04.516440,2024-04-09 17:44:53.662875,0.000013,16,8,3,932
1,1,0.043519,0.989534,0.047869,2024-04-09 17:44:53.704285,2024-04-09 22:05:30.327186,0.000031,4,6,4,1067
2,2,0.054714,0.987621,0.062322,2024-04-09 22:05:30.359980,2024-04-10 00:26:13.613859,0.000013,16,4,2,1033
3,3,0.051484,0.987701,0.051460,2024-04-10 00:26:13.647323,2024-04-10 03:45:18.480474,0.000016,7,6,3,725
4,4,0.055351,0.986182,0.051536,2024-04-10 03:45:18.521370,2024-04-10 06:06:12.163497,0.000021,5,11,2,492
...,...,...,...,...,...,...,...,...,...,...,...
131,131,0.038389,0.990733,0.048764,2024-05-06 06:27:25.063577,2024-05-06 13:46:27.471692,0.000031,11,9,7,678
132,132,0.037077,0.990888,0.049814,2024-05-06 13:46:27.504890,2024-05-06 20:03:11.129133,0.000025,29,9,6,560
133,133,0.032777,0.991232,0.054029,2024-05-06 20:03:11.173119,2024-05-07 03:21:55.932267,0.000029,32,9,7,654
134,134,0.034727,0.991043,0.050602,2024-05-07 03:21:55.974912,2024-05-07 10:41:11.549722,0.000025,32,9,7,656


In [10]:
find_best_df = study_results_df.drop(labels='datetime_start', axis=1)
find_best_df = find_best_df.drop(labels='datetime_complete', axis=1)

In [11]:
# # Export whole Optuna results
# study_results_df.round(6).to_csv(f"OptunaResults.csv", index=True)

## Get Top-Runs per Objective

In [12]:
dataframes = []
round_decimals = 6
top_number = 10

In [13]:
## LOSS ascending
# sort df after values_0
study_results_df_loss = study_results_df.sort_values(by='values_0', ascending=True).head(top_number).round(round_decimals)
study_results_df_loss.head()
dataframes.append(study_results_df_loss)


In [14]:
## BLEU descending
# sort df after values_0
study_results_df_bleu = study_results_df.sort_values(by='values_1', ascending=False).head(top_number).round(round_decimals)
study_results_df_bleu.head()
dataframes.append(study_results_df_bleu)


In [15]:
## ROUGE descending
# sort df after values_0
study_results_df_rouge = study_results_df.sort_values(by='values_2', ascending=False).head(top_number).round(round_decimals)
study_results_df_rouge.head(30)
dataframes.append(study_results_df_rouge)

In [16]:
# # Export Dataframes to CSV
# for id, df in enumerate(dataframes):
#     df.T.to_csv(f"transposed_top{top_number}_objective_{id}.csv", index=True)
#     df.T.to_csv(f"transposed_top{top_number}_objective_{id}_float.csv", float_format=f"%.{round_decimals}f", index=True)


In [17]:
dataframes[2].head()

,number,values_0,values_1,values_2,datetime_start,datetime_complete,params_learning_rate,params_lora_alpha_value,params_lora_r_value,params_num_train_epochs,params_warmup_steps
2,2,0.054714,0.987621,0.062322,2024-04-09 22:05:30.359980,2024-04-10 00:26:13.613859,0.000013,16,4,2,1033
130,130,0.038864,0.990470,0.061189,2024-05-06 00:09:35.306738,2024-05-06 06:27:25.035615,0.000030,11,9,6,675
42,42,0.044770,0.989132,0.059830,2024-04-15 08:07:16.840544,2024-04-15 10:28:49.325439,0.000031,12,11,2,852
88,88,0.047143,0.988618,0.059485,2024-04-22 16:10:44.243927,2024-04-22 18:32:04.552728,0.000028,10,8,2,593
39,39,0.037798,0.990522,0.058879,2024-04-14 20:05:56.711789,2024-04-15 01:24:41.843686,0.000035,13,8,5,1098


In [18]:
# df
find_best_df

find_best_df_ws = find_best_df

## Weighted Sum Calculation

In order to define which of the hyperparameter tuning trials was the best in the multi-object optimization and thus define the optimal hyperparameters, a weighted sum was defined. The objectives BLEU and LOSS were used for the weighted sum, whereby the LOSS was defined as an inverse loss $\widetilde{L}(x)$ with $1-loss$.

Concluding to the Formular:

$$w_{\text{Loss}} =  w_{\text{BLEU}} = 0.5$$
$$f(x) = w_{\text{Loss}} \cdot \widetilde{L}(x) +  w_{\text{BLEU}} \cdot B(x)$$

In [25]:
# add inverted Loss
find_best_df_ws['inverted_loss'] = 1 - find_best_df_ws['values_0']

find_best_df_ws['weighted_sum'] = (0.5 * find_best_df_ws['inverted_loss']) + (0.5 * find_best_df_ws['values_1'])

# Sort the DataFrame based on the 'weighted_sum' column in descending order
find_best_df_ws_sorted = find_best_df_ws.sort_values(by='weighted_sum', ascending=False)

# If you want to reset the index after sorting
find_best_df_ws_sorted = find_best_df_ws_sorted.reset_index(drop=True)

# Display the sorted DataFrame
find_best_df_ws_sorted.head()

,number,values_0,values_1,values_2,params_learning_rate,params_lora_alpha_value,params_lora_r_value,params_num_train_epochs,params_warmup_steps,inverted_loss,weighted_sum
0,116,0.031224,0.991329,0.046125,0.000034,30,10,6,448,0.968776,0.980053
1,127,0.031686,0.991554,0.046529,0.000042,22,5,6,1044,0.968314,0.979934
2,124,0.032367,0.991696,0.047422,0.000034,24,10,9,974,0.967633,0.979664
3,108,0.031922,0.991245,0.048796,0.000037,18,5,8,911,0.968078,0.979662
4,102,0.032023,0.991332,0.048186,0.000037,16,6,8,917,0.967977,0.979654


## Hyperparameter Importance

The importance of the hyperparameters for the outcome of the objectives was determined using a random forest regressor. 

The importance provides information about which hyperparameters should possibly no longer be defined as hyperparameters in future versions.

In [22]:
df_imp_self = study.trials_dataframe()

df_imp_self.drop(105, inplace=True)

df_imp_self

,number,values_0,values_1,values_2,datetime_start,datetime_complete,duration,params_learning_rate,params_lora_alpha_value,params_lora_r_value,params_num_train_epochs,params_warmup_steps,system_attrs_nsga3:generation,state
0,0,0.047838,0.988572,0.049686,2024-04-09 14:26:04.516440,2024-04-09 17:44:53.662875,0 days 03:18:49.146435,0.000013,16,8,3,932,0.0,COMPLETE
1,1,0.043519,0.989534,0.047869,2024-04-09 17:44:53.704285,2024-04-09 22:05:30.327186,0 days 04:20:36.622901,0.000031,4,6,4,1067,0.0,COMPLETE
2,2,0.054714,0.987621,0.062322,2024-04-09 22:05:30.359980,2024-04-10 00:26:13.613859,0 days 02:20:43.253879,0.000013,16,4,2,1033,0.0,COMPLETE
3,3,0.051484,0.987701,0.051460,2024-04-10 00:26:13.647323,2024-04-10 03:45:18.480474,0 days 03:19:04.833151,0.000016,7,6,3,725,0.0,COMPLETE
4,4,0.055351,0.986182,0.051536,2024-04-10 03:45:18.521370,2024-04-10 06:06:12.163497,0 days 02:20:53.642127,0.000021,5,11,2,492,0.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,131,0.038389,0.990733,0.048764,2024-05-06 06:27:25.063577,2024-05-06 13:46:27.471692,0 days 07:19:02.408115,0.000031,11,9,7,678,NaN,COMPLETE
132,132,0.037077,0.990888,0.049814,2024-05-06 13:46:27.504890,2024-05-06 20:03:11.129133,0 days 06:16:43.624243,0.000025,29,9,6,560,NaN,COMPLETE
133,133,0.032777,0.991232,0.054029,2024-05-06 20:03:11.173119,2024-05-07 03:21:55.932267,0 days 07:18:44.759148,0.000029,32,9,7,654,NaN,COMPLETE
134,134,0.034727,0.991043,0.050602,2024-05-07 03:21:55.974912,2024-05-07 10:41:11.549722,0 days 07:19:15.574810,0.000025,32,9,7,656,NaN,COMPLETE


In [24]:
df_imp_self = study.trials_dataframe()
df_imp_self.drop(105, inplace=True)



def plot_param_importance(df, objective_cols=['values_0'], objective_names=['VALUE_NAME']):
    combined_importance_df = pd.DataFrame(columns=['Parameter'])
    
    for objective_col in objective_cols:
        # Extract the relevant columns
        param_cols = [col for col in df.columns if col.startswith('params_')]
        X = df[param_cols]
        y = df[objective_col]

        # Train a Random Forest model
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X, y)

        # Calculate permutation importances
        result = permutation_importance(model, X, y, n_repeats=10, random_state=42, n_jobs=-1)

        # Create a DataFrame for plotting
        importance_df = pd.DataFrame({
            'Parameter': X.columns,
            f'Importance_{objective_col}': result.importances_mean
        })

        # Sort by importance
        importance_df = importance_df.sort_values(by='Parameter')

        # Rename parameters
        importance_df['Parameter'] = importance_df['Parameter'].replace({
            'params_lora_r_value': 'lora_r_value',
            'params_lora_alpha_value': 'lora_alpha_value',
            'params_warmup_steps': 'warmup_steps',
            'params_num_train_epochs': 'num_train_epochs',
            'params_learning_rate': 'learning_rate'
        })

        # Merge with combined_importance_df
        combined_importance_df = pd.merge(combined_importance_df, importance_df, on='Parameter', how='outer')

    # Plot using Plotly
    fig = go.Figure()

    for id, objective_col in enumerate(objective_cols):
        fig.add_trace(go.Bar(x=combined_importance_df['Parameter'], y=combined_importance_df[f'Importance_{objective_col}'], name=f'{objective_names[id]}'))

    fig.update_layout(barmode='group', title='Parameter Importance for Multiple Objectives', xaxis_title='Hyperparameter', yaxis_title='Permutation Importance')
    fig.update_layout(legend_title="Objectives")
    fig.update_traces(texttemplate='%{y:.2f}', textposition='auto')

    ## Export Image
    # fig.write_image("./plotly/ParameterImportanceforMultipleObjectives_800_400_5.png", width=800, height=400, scale=5)
    
    
    fig.show()
# (Importance calculated using permutation importance with random forrest regressor)
# Example usage
# Assuming 'df' is your DataFrame loaded from the study
# df = pd.read_csv('your_dataframe.csv')  # Load your DataFrame
plot_param_importance(df_imp_self, objective_cols=['values_0', 'values_1', 'values_2'], objective_names=['Loss','BLEU','ROUGE'])


**More evaluations and overviews can be found in the Optuna Dashboard. See README.md for further instructions.**